In [4]:
import pandas as pd

df =pd.read_csv("../dataset/romanian_political_articles_v1.csv")

In [5]:
df.isnull().sum()
df = df.dropna(subset=["maintext", "source_domain"])

In [6]:
import re
import spacy
from unidecode import unidecode

nlp = spacy.load("ro_core_news_sm")

def remove_quotes(text):
    return re.sub(r'"[^"]+"', '', text)

df['cleantext'] = df['maintext'].apply(remove_quotes)

def preprocess_text(text : str):
    if not text or len(text.strip()) == 0:
        return ""

    text = remove_quotes(text)

    # Remove all punctuation except periods
    text = re.sub(r'[^\w\s.]', '', text)

    # Remove digits and numbers
    text = re.sub(r'\b\d+\b', '', text)

    text = re.sub(r'\.\s+', ' PERIOD ', text)
    if text.endswith('.'):
        text = text[:-1] + ' PERIOD'

    doc = nlp(text)
    sentences = []
    for sent in doc.sents:
        words = sent.text.split()
        if words:
            words[0] = words[0].lower()
            sentences.append(' '.join(words))
    cleaned = ' '.join(sentences)

    return cleaned

df['cleantext'] = df['maintext'].apply(preprocess_text)

df = df[df['cleantext'].str.split().str.len() > 100]

df.head()

,url,title,date_publish,description,maintext,source_domain,authors,cleantext
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,activistă și ea din zona soroșistă ce a venit ...
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,crin Antonescu Ei promit lapte și miere dar nu...
3,https://www.realitatea.net/stiri/politica/lasc...,Lasconi: „Nicușor Dan mi-a cerut să mă retrag ...,2025-04-12 19:02:32,",,Nicusor Dan mi-a cerut sa ma duc la Stejarii...",Candidata la prezidențiale trădată de propriul...,www.realitatea.net,Georgiana Balaban,candidata la prezidențiale trădată de propriul...
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,totul pentru a evita o situație asemănătoare c...
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,actorul spune că românii sunt îngenuncheați și...


In [ ]:
def extract_ngrams_spacy(text, n):
    doc = nlp(text)
    tokens = [
        token.text for token in doc
        if not token.is_space and not token.is_punct and token.is_alpha
    ]

    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]

df['monograms'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 1))
df['bigrams'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 2))
df['trigrams'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 3))
df['all_ngrams'] = df['bigrams'] + df['trigrams'] + df['mono']
df.head()

,url,title,date_publish,description,maintext,source_domain,authors,cleantext,monograms,bigrams,trigrams,all_ngrams
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,activistă și ea din zona soroșistă ce a venit ...,"[(activistă,), (și,), (ea,), (din,), (zona,), ...","[(activistă, și), (și, ea), (ea, din), (din, z...","[(activistă, și, ea), (și, ea, din), (ea, din,...","[(activistă, și), (și, ea), (ea, din), (din, z..."
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,crin Antonescu Ei promit lapte și miere dar nu...,"[(crin,), (Antonescu,), (Ei,), (promit,), (lap...","[(crin, Antonescu), (Antonescu, Ei), (Ei, prom...","[(crin, Antonescu, Ei), (Antonescu, Ei, promit...","[(crin, Antonescu), (Antonescu, Ei), (Ei, prom..."
3,https://www.realitatea.net/stiri/politica/lasc...,Lasconi: „Nicușor Dan mi-a cerut să mă retrag ...,2025-04-12 19:02:32,",,Nicusor Dan mi-a cerut sa ma duc la Stejarii...",Candidata la prezidențiale trădată de propriul...,www.realitatea.net,Georgiana Balaban,candidata la prezidențiale trădată de propriul...,"[(candidata,), (la,), (prezidențiale,), (trăda...","[(candidata, la), (la, prezidențiale), (prezid...","[(candidata, la, prezidențiale), (la, preziden...","[(candidata, la), (la, prezidențiale), (prezid..."
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,totul pentru a evita o situație asemănătoare c...,"[(totul,), (pentru,), (a,), (evita,), (o,), (s...","[(totul, pentru), (pentru, a), (a, evita), (ev...","[(totul, pentru, a), (pentru, a, evita), (a, e...","[(totul, pentru), (pentru, a), (a, evita), (ev..."
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,actorul spune că românii sunt îngenuncheați și...,"[(actorul,), (spune,), (că,), (românii,), (sun...","[(actorul, spune), (spune, că), (că, românii),...","[(actorul, spune, că), (spune, că, românii), (...","[(actorul, spune), (spune, că), (că, românii),..."


In [ ]:
from collections import Counter
import numpy as np

all_phrases = list(set(phrase for phrases_list in df['all_ngrams'] for phrase in phrases_list))

# Create the Nij matrix (rows: phrases, columns: domains)
domains = df['source_domain'].unique()
nij_matrix = pd.DataFrame(0, index=all_phrases, columns=domains)

# Fill the Nij matrix
for domain in domains:
    domain_articles = df[df['source_domain'] == domain]
    domain_phrases = Counter()

    for phrases_list in domain_articles['all_ngrams']:
        domain_phrases.update(phrases_list)

    for phrase, count in domain_phrases.items():
        nij_matrix.at[phrase, domain] = count

In [ ]:
monograms = set(mono for monograms_list in df['monograms'] for mono in monograms_list)
bigrams = set(bi for bigrams_list in df['bigrams'] for bi in bigrams_list)
trigrams = set(tri for trigrams_list in df['trigrams'] for tri in trigrams_list)

def is_subgram(sub, full):
    n = len(sub)
    return any(full[i:i+n] == sub for i in range(len(full) - n + 1))

def find_phrases_to_delete(shorter_phrases, longer_phrases, nij_matrix, threshold=0.7):
    """
    Identify phrases from 'shorter_phrases' to delete if they mostly appear inside any single 'longer_phrase'.

    Args:
        shorter_phrases (set): monograms or bigrams
        longer_phrases (set): bigrams or trigrams
        nij_matrix (DataFrame): phrase x domain counts
        threshold (float): fraction threshold to delete

    Returns:
        list: phrases to delete
    """
    mapping = {short: [] for short in shorter_phrases}

    for long_phrase in longer_phrases:
        for short in shorter_phrases:
            if is_subgram(short, long_phrase):
                mapping[short].append(long_phrase)

    phrases_to_delete = []

    for short, long_list in mapping.items():
        short_count = nij_matrix.loc[short].sum()

        for long_phrase in long_list:
            long_count = nij_matrix.loc[long_phrase].sum()
            if long_count / short_count >= threshold:
                phrases_to_delete.append(short)
                break  # No need to check further if one passes

    return phrases_to_delete

# monograms_to_delete = find_phrases_to_delete(monograms, bigrams, nij_matrix, threshold=0.7)
# monograms_to_delete.append(find_phrases_to_delete(monograms, trigrams, nij_matrix, threshold=0.7))
# bigrams_to_delete = find_phrases_to_delete(bigrams, trigrams, nij_matrix, threshold=0.7)

# print(monograms_to_delete)

True
